In [ ]:
# CNN Data Prep
x_train_cnn = x_train
x_test_cnn = x_test

x_train_cnn['fill1'] = 0
x_test_cnn['fill1'] = 0

print(x_train_cnn.shape)
x_train_reshaped = x_train_cnn.values.reshape(7008, 5, 3)
print(x_train_reshaped.shape)

print(x_test_cnn.shape)
x_test_reshaped = x_test_cnn.values.reshape(1752, 5, 3)
print(x_test_reshaped.shape)

num_train, height, width = x_train_reshaped.shape

In [ ]:
import tensorflow as tf

# callback to stop when there is no loss improvement
callback = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=10)

inp = Input(shape=(height, width, 1)) 

conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

flat = Flatten()(drop_1)
hidden = Dense(hidden_size)(flat)
out = Dense(output_size)(hidden)

# To define a model, just specify its input and output layers 
model = Model(inputs=inp, outputs=out) 

# using the cross-entropy loss function 
model.compile(loss='mse', 
            optimizer='adam', 	    # using the Adam optimiser
            metrics=['mse']) # reporting the MSE and accuracy

# Train the model using the training set
# validation_split indicates using fraction 0.1 (10%) for validation
history = model.fit(x_train_reshaped, y_train,                
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1,
          callbacks=[callback])

model.evaluate(x_test_reshaped, y_test, verbose=1)  # Evaluate the trained model on the test set!

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix

y_pred = model.predict(x_test_reshaped)

r2_value = r2_score(y_test, y_pred.round())
print("r2")
print(r2_value)

rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred.round()))
print("mse")
print(rmse_lstm)

In [ ]:
import matplotlib.pyplot as plt

fig1, ax1 = plt.subplots()
line1 = ax1.plot(history.history["loss"])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss")

fig2, ax2 = plt.subplots()
line2 = ax2.plot(history.history["mse"])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("MSE")

fig3, ax3 = plt.subplots()
line3 = ax3.plot(list(y_test)[:50], label='true')
line4 = ax3.plot(y_pred[:50], label='predicted')
ax3.legend()
ax3.set_xlabel("Time")
ax3.set_ylabel("Rented Bike Count")